### 所有子数组最小值的累加和
https://leetcode.cn/problems/sum-of-subarray-minimums/

In [ ]:
'''
思路：
以x位置为最小值，左右找到临界边界。
左边界(不包含)到当前值(包含)有m个，右边界(不包含)到当前值(包含)有n个数，
所以有m*n个数以x位置为最小值。所以产生m*n*arr[x]累加和

当有重复值时，右边界为全局临界，左边界从上一个相等数的位置开始。（为了去重）
或者：       左边界为全局邻居，右边界只停到相等数的位置。
'''

def subArrayMinSum(arr):
    #left[i] = x: arr[i]左边离arr[i]最近，比arr[i]小或等于的数，位置为x。没有这样的数为-1
    left = leftNearLessEqual(arr)
    #right[i] = x: arr[i]右边离arr[i]最近，比arr[i]小的数，位置为x。没有这样的数为-1
    right = leftNearLess(arr)
    ans = 0
    for i in range(len(arr)):
        start = i - left[i]
        end = right[i] - i
        ans += start * end * arr[i]
    return ans

### 求斐波那契数列矩阵乘法的方法
1. F(N) = F(N-1) + F(N-2), 严格依赖前两项递推，线性求解O(N)
2. F(N) = ... 严格依赖前项且没有条件转移，都有O(logN)的最优解法。 斐波那契数为二阶递
3. |F(N), F(N-1)| = |F(2), F(1)| * (某个矩阵)^{n-2}, ==> 矩阵的n次方怎么算最快 ==> n的x次方怎么算最快


In [13]:
'''
思路：
一个数n的x次方，x拆成2进制。每次x都乘自己，那么就会得到x,x^2,x^4...。
观察x的2进制都需要哪些次方，从base=1开始，每次乘上前面得到的x次方。
实际是利用2进制，进行二分法。
================
一个矩阵的x次方，从base=单位矩阵开始，每次乘上得到的矩阵次方。
'''
#O(logN)
import numpy as np
def f(n):
    if n < 1:
        return 0
    if n == 1 or n == 2:
        return 1
    base = np.array([[1,1],[1,0]]) #|a,b|
                                   #|c,d| 由推导得出
    res = matrixPower(base, n-2)
    return res[0][0] + res[1][0] #|Fn,Fn-1| = |F2,F1| * |a,b| ^ (n-2)
                                 #                      |c,d|
                                 #相当于     = |1,1|   * |a,b| ^ (n-2)
                                 #                      |c,d|
                                 #只要Fn时，即只要a+c部分
                
def matrixPower(m, power):
    res = [[0] * len(m) for _ in range(len(m[0]))]
    for i in range(len(res)):
        res[i][i] = 1   #单位矩阵
    t = m #矩阵1次方
    while power != 0:
        if power & 1 != 0:
            res = multiMatrix(res, t)
        t = multiMatrix(t, t)
        power >>= 1
    return res

def multiMatrix(m1, m2):
    res = [[0] * len(m1) for _ in range(len(m2[0]))]
    for i in range(len(m1)):
        for j in range(len(m2[0])):
            for k in range(len(m2)):
                res[i][j] += m1[i][k] * m2[k][j]
    return res

In [18]:
f(7)

13

In [7]:
base

array([[1, 1],
       [1, 0]])

In [9]:
np.dot(base, np.array([[3,2],[2,3]]))

array([[5, 5],
       [3, 2]])

In [11]:
multiMatrix(base, np.array([[3,2],[2,3]]))

[[5, 5], [3, 2]]

### 递推公式推广
Fn = a*Fn-1 + b*Fn-2.. + k*F(n-i) 都有O(logN)的解法

In [19]:
'''
F(n) = 7*F(n-1) - 3*F(n-2) + 4*F(n-3) #三阶递推公式(n-3,最末尾的依赖n-i，i是几就是几阶)
|F4,F3,F2| = |F3,F2,F1| * | 3*3 | 
|Fn,Fn-1,Fn-2| = |F3,F2,F1| * | 3*3 | ^ (n-3)

==> 推广

|Fn,...Fn-i| = |Fi...F1| * |i*i| ^ (n-i)
'''

#### 母牛生小牛, N年后牛的数量
第一年农场有1只成熟的母牛A，往后的每年：
1. 每年成熟的母牛都会生一只母牛
2. 每一只新出生的母牛都会在出生的第三年成熟
3. 每一只母牛永远不会死
4. 返回N年后的牛的数量

In [20]:
'''
第一年：A(1) = 1
第二年：A(2),B(0->A) = 2
第三年：A(3),B(1),C(0->A) = 3
第四年：A(4),B(2),C(1),D(0->A) = 4
第五年：A(5),B(3),C(2),D(1),E(0->A),F(0->B) = 6
第六年：A(6),B(4),C(3),D(2),E(1),F(1),G(0->A),H(0->B),I(0->C) = 9
==> Fn = Fn-1 + Fn-3

|F4F3F2| = |F3F2F1| * |abc|
                      |def|
                      |ghi|
                      
=> |4,3,2| = |3,2,1|*|abc| ==> 3a+2d+g=4; 3b+2e+h=3; 3g+2h+i = 2
                     |def|
                     |ghi|
=> |6,4,3| = |4,3,2|*|abc| ==> 4a+3d+2g=6; 4b+3e+2h=4; 4g+3h+2i = 3
                     |def|
                     |ghi|
=> |9,6,4| = |6,4,3|*|abc| ==> 6a+4d+3g=9; 6b+4e+3h=6; 6g+4h+3i = 4
                     |def|
                     |ghi|
==> |abc| = |110|
    |def|   |001|
    |ghi|   |100|
    
所以 |FnFn-1Fn-2| = |321| * |3*3| ==> Fn = 3a+2d+g
'''
def c3(n):
    if n < 1:
        return 0
    if n == 1 or n==2 or n==3:
        return n
    base = np.array([[1,1,0],[0,0,1],[1,0,0]])
    res = matrixPower(base, n-3)
    return 3 * res[0][0] + 2 * res[1][0] + res[2][0]

In [21]:
c3(19)

1278

#### 由0和1两种字符构成的达标字符串
* 给定一个数N，想象为只有01两种字符组成，长度为N字符串。如果某个字符串，任何0字符的左边都有1紧挨着，认为这个字符串达标。
* 返回有多少达标的字符串

In [ ]:
'''
N = 1, 0❌,1✅ ==> 1个
N = 2, 00❌,01❌,10✅,11✅ = 2个
N = 3, 000❌, 001❌, 010❌, 100❌, 011❌, 101✅, 110✅, 111✅ = 3个
方法1:观察得到 1，2，3，5，8，13 ==> F1=1，F2=2，Fn=Fn-1+Fn-2
=======
方法2:
定义f(i)-> 第一个数为1，后续有i个数要填，能有多少种达标。如果后续第一个格子填1，则满足f(i-1)。如果填0，则需要跟一个1，所以满足f(i-2)
所以f(i) = f(i-1) + f(i-2)
所以N=6时，调f(5)。

方法2改矩阵乘法。
'''

#### 铺瓷砖
给定N，有2 x N的区域铺瓷砖，一块瓷砖为1 x 2. 问需要多少种摆法

In [ ]:
'''
f(n) 代表2*n区域要摆。
如果第一列，竖着摆，则fn变为f(n-1)问题；
如果前两列，横着摆，则变成f(n-2)问题
'''